In [1]:
import numpy as np

This evaluates the circuit 
<img src="./images/common_emitter_amplifier.PNG" />

In [21]:
def eval_common_emtter_amp_circuit(val):
    Vdc_th = val['Vcc'] * (val['R2']/(val['R1'] + val['R2']))
    Rb_th = 1.0 / (1.0/val['R1'] + 1.0/val['R2'])    

    #KVL
    #Ie = (beta + 1)*Ib
    # Vth - Rth*Ib - Vbe - Re*Ie 
    #= Vth - Rth*Ib - Vbe - Re*(beta + 1)*Ib
    #Vth - Vbe  = Rth*Ib + Re*(beta + 1)*Ib
    #Ib = (Vth - Vbe) / (Rth + Re*(beta + 1))
    Ib = (Vdc_th - val['Vbe']) / (Rb_th + val['Re']*(val['beta'] + 1))
    Ic = val['beta'] * Ib
    Ie = (val['beta'] + 1) * Ib
    Vce = val['Vcc'] - Ic*val['Rc'] - Ie*val['Re']
    Ic_saturated = val['Vcc']/(val['Rc'] + val['Re'])
    Vce_off = val['Vcc']
    dc_analysis = {
    #Compute the parallel values for the base
    'Vdc_th': Vdc_th,
    'Rb_th': Rb_th,

    #KVL
    #Ie = (beta + 1)*Ib
    # Vth - Rth*Ib - Vbe - Re*Ie 
    #= Vth - Rth*Ib - Vbe - Re*(beta + 1)*Ib
    #Vth - Vbe  = Rth*Ib + Re*(beta + 1)*Ib
    #Ib = (Vth - Vbe) / (Rth + Re*(beta + 1))
    'Ib': Ib,
    'Ic': Ic,
    'Ic_saturated': Ic_saturated,
    'Ie': Ie,
    'Vce': Vce,
    'Vce_off': Vce_off,
    }
    
    re = 26e-3 / Ie
    
    
    #Avoc = Vout/Vin = -Rc*ic/(re*ie) = -Rc*beta*ib/(re*(1+beta)*ib) 
    #Avoc = -Rc/re
    Avoc = (-val['Rc']*(val['beta'])) / (re*(val['beta']+1))
    
    #ZinQ = Vin /iin = re*ie/ib =  re*(beta+1)*ib/ib = re*(beta+1)    
    ZinQ = re*(val['beta']+1)
    
    #Zin = Rb_th || ZinQ
    Zin = 1/(1/Rb_th +1/ZinQ)
    
    Zout = val['Rc']
    ac_analysis = {'re': re, 'Avoc': Avoc, 'ZinQ':ZinQ }
    amplifier = {'Zin': Zin, 'Zout': Zout, 'gain': Avoc}

    analysis = {'dc': dc_analysis, 'ac': ac_analysis, 'amplifier': amplifier}
    
    
    return analysis

In [28]:

common_emtter_amp_circuit = {
    'Vcc': 10, 
    'R1': 16e3,
    'R2': 2.4e3,
    'Rc': 1e3,
    'Re': 100,
    #This circuit is fairly indepedent of beta (100-300)
    'beta': 300,
    'Vbe': 0.7
}

#Loop through betas
analysis = eval_common_emtter_amp_circuit(common_emtter_amp_circuit)

RLoad=1000
print(f"Ib = {analysis['dc']['Ib']:0.3e}A, Ic = {analysis['dc']['Ic']:.3e}A")
print(f"Vth = {analysis['dc']['Vdc_th']:0.2f}V, Rb_th = {analysis['dc']['Rb_th']:.0f} ohms")   
print(f"The Q point is Vce = {analysis['dc']['Vce']:0.2f}V, Ic = {analysis['dc']['Ic']:.3e}A")
print(f"re = {analysis['ac']['re']:0.3e} ohm")
      
# The reason the circuit 
print(f"The predicted open load voltage gain is {analysis['amplifier']['gain']} or {20*np.log10(np.abs(analysis['amplifier']['gain']))} dB")
      

Ib = 1.878e-05A, Ic = 5.633e-03A
Vth = 1.30V, Rb_th = 2087 ohms
The Q point is Vce = 3.80V, Ic = 5.633e-03A
re = 4.600e+00 ohm
The predicted opn load voltage gain is -216.648136410395 or 46.71509915241476 dB


In [ ]:
#